# --- Day 24: Crossed Wires ---

In [294]:
# --- Part One and Part Two ---

from collections import defaultdict,Counter
from typing import List, Dict, Tuple, Set
# from functools import cache
from itertools import chain

filename = 'input.txt'
#filename = 'test1.txt' # decomment for test
#filename = 'test2.txt' # decomment for test

a = None
wires = defaultdict(None)
signals = {}

def logic_execute(logic: Tuple)-> bool:
    global wires
    outn, (an, fn, bn) = logic
    a = wires[an]
    b = wires[bn]
    out = wires[outn]
    if a is None or b is None:
        return False
    if fn == "AND":
        out = a & b
    elif fn == "OR":
        out = a | b
    elif fn == "XOR":
        out = a ^ b 
    else:
        return False
    wires[outn] = out
    return True

def parse_file(filename):
    global wires
    global signals
    with open(filename, 'r') as file:
        lines = [l.strip() for l in file]
        # get signals/wires list
        i = lines.index('')
        wires = {k.strip() : int(v.strip()) for k, v in  [l.split(':') for l in lines[:i]]}
        # logic_ports = {tuple([*lg, out]) : out for lg, out in [(logic.strip().split(' '), o) for logic, o in  [l.split('-> ') for l in lines[i+1:]]]}
        signals = { out : tuple(lg) for lg, out in [(logic.strip().split(' '), o) for logic, o in  [l.split('-> ') for l in lines[i+1:]]]}
        #print (logic_ports)
        for s in signals:  
            wires[s] = None

def execute():
    global signals
    queue = [l for l in signals.items()]
    while queue:
        l = queue.pop(0)
        if not logic_execute(l):
            queue.append(l)


# --- main ---

parse_file(filename)

execute()

z = "0b"+''.join([str(c) for k, c in sorted(wires.items()) if k.startswith('z') ][::-1])
print(z)
res1 =  int(z,2)

print(f"The solution for part 1 is: {res1}")



0b1101000110000001000100111011111110001101001110
The solution for part 1 is: 57588078076750


In [295]:

def wires_get(v):
    """return int value of bits v00 - vxx from wires"""
    variable = "0b"+''.join([str(c) for k, c in sorted(wires.items()) if k.startswith(v[0])][::-1])
    return int(variable,2)

def wires_set(v, value): # force 45 bits
    for i in range(45):
        variable = f"{v[0]}{i:02}"
        wires[variable] = value & 1
        value = value >> 1

def evaluate(x, y) -> int:
    global wires
    # reset all signals
    for w in wires:
        wires[w] = None
    wires_set('x',x)
    wires_set('y',y)
    execute()
    return(wires_get('z'))

def touched(v):
    global signals
    retlist = set()
    """ return list of signals (excluding x anf y) that are involvend in 'v' calculation """
    if v[0] in ['x', 'y']:
        return list(retlist)
    queue = [v]
    while queue:
        s = queue.pop(0)
        retlist.add(s)
        s1, _, s2 = signals[s]
        if s1[0] not in ['x', 'y'] and s1 not in retlist:
            queue.append(s1)
        if s2[0] not in ['x', 'y'] and s2 not in retlist:
            queue.append(s2)
    return list(retlist)

# find failed bits fron z

#t = sorted(touched('z31'))

#print (t)

# execute with default x and y
execute()

y = wires_get('y')
x = wires_get('x')
z = wires_get('z') 

print(x, y, z, x + y)

mask = z ^ (x + y)
failed_bits = [f"z{i:02}" for i,v in enumerate(bin(mask)[2:][::-1]) if v == '1']

print (failed_bits)

# get signals involved in all failed_bits
# involved = [touched(fb) for fb in failed_bits]

# print (involved)
# print (len(involved))

# all_involved = list(chain(*involved))
# print (all_involved)


#all_involved_reduced = set(all_involved[0]) & set(all_involved[1])
# for l in all_involved[1:]:
#     all_involved_reduced = all_involved_reduced & set(l)

bad_z =[]
for j in range(45):
    x = 0
    y = 1 << j
    if evaluate(x, y) != x + y:
        
        bad_z.append(f"z{j:02}")

print(bad_z)  



28885735145135 28702208779295 57588078076750 57587943924430
['z07', 'z08', 'z16', 'z17', 'z18', 'z19', 'z20', 'z27', 'z28', 'z29', 'z30']
['z07', 'z16', 'z23', 'z27']


For the solution of part 2 I proceed by hand using notes.txt and the following code

In [296]:
v = "wvb"
a, f, b = signals[v]
f"{v} = {f} ({a} {b})"

'wvb = AND (x26 y26)'

In [304]:
# --- Part Two ---

# Test if correct solution

parse_file(filename)

swaps = [("z07" , "shj"), ("tpk", "wkb"), ("z23", "pfn"), ("z27", "kcd")]

for sw in swaps:
    a = signals[sw[0]]
    b = signals[sw[1]]
    signals[sw[0]] = b
    signals[sw[1]] = a

error = False
for i in range(45):
    for j in range(45):
        x = 1 << i
        y = 1 << j
        if evaluate(x, y) != x + y:
            print (f"Something is not working")
            error = True
            break
    if error : break

if not error:
    print ("FANTASTIC")
    res2 = ','.join(sorted(list(chain(*[[*l] for l in swaps ]))))


print(f"The solution for part 2 is: {res2}")

FANTASTIC
The solution for part 2 is: kcd,pfn,shj,tpk,wkb,z07,z23,z27
